In [1]:
# %%bash
# SRC="/kaggle/input/verse-19-3d-images/dataset-verse19validation"
# DST="/kaggle/working/verse19/dataset-verse19validation"

# mkdir -p "$DST/rawdata" "$DST/derivatives"

# # Get first 5 folders and copy them
# for sub in $(ls -1 "$SRC/rawdata" | head -n 5); do
#     cp -r "$SRC/rawdata/$sub" "$DST/rawdata/"
#     cp -r "$SRC/derivatives/$sub" "$DST/derivatives/"
#     echo "Copied $sub"
# done

In [2]:
# %%bash
# SRC="/kaggle/input/verse-19-3d-images/dataset-verse19training"
# DST="/kaggle/working/verse19/dataset-verse19training"

# mkdir -p "$DST/rawdata" "$DST/derivatives"

# # Get first 5 folders and copy them
# for sub in $(ls -1 "$SRC/rawdata" | head -n 5); do
#     cp -r "$SRC/rawdata/$sub" "$DST/rawdata/"
#     cp -r "$SRC/derivatives/$sub" "$DST/derivatives/"
#     echo "Copied $sub"
# done

In [3]:
# %%bash
# SRC="/kaggle/input/verse-19-3d-images/dataset-verse19test"
# DST="/kaggle/working/verse19/dataset-verse19test"

# mkdir -p "$DST/rawdata" "$DST/derivatives"

# # Get first 5 folders and copy them
# for sub in $(ls -1 "$SRC/rawdata" | head -n 5); do
#     cp -r "$SRC/rawdata/$sub" "$DST/rawdata/"
#     cp -r "$SRC/derivatives/$sub" "$DST/derivatives/"
#     echo "Copied $sub"
# done

In [4]:
# !rm -rf /kaggle/working/SpineSegmentation_HealthiVert
!git clone https://github.com/ahmed-226/SpineSegmentation_HealthiVert.git

Cloning into 'SpineSegmentation_HealthiVert'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 84 (delta 29), reused 74 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 77.30 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [5]:
!cd /kaggle/working/SpineSegmentation_HealthiVert/ && ! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
! cd SpineSegmentation_HealthiVert/ && ! python prepare_data.py --data_dir /kaggle/input/verse-19-3d-images --output_dir .

Scanning for subjects in /kaggle/input/verse-19-3d-images...
Found 136 subjects.
Split distribution: {'validation': 36, 'test': 35, 'training': 65}
Subjects with landmarks: 136
Subjects with segmentation masks: 135

Final splits:
  Training: 65
  Validation: 36
  Test: 35
Traceback (most recent call last):
  File "/kaggle/working/SpineSegmentation_HealthiVert/prepare_data.py", line 252, in <module>
    main()
  File "/kaggle/working/SpineSegmentation_HealthiVert/prepare_data.py", line 218, in main
    with open(train_file, 'w') as f:
         ^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 30] Read-only file system: '/kaggle/input/verse-19-3d-images/train.txt'


In [7]:
# !rm -rf verse19/output
# !mkdir -p verse19/output

In [ ]:
!cd SpineSegmentation_HealthiVert/ && python train_kfold.py \
    --data_dir /kaggle/input/verse-19-3d-images \
    --output_dir /kaggle/working/output \
    --splits_dir . \
    --stage all \
    --n_folds 5 \
    --iterations 10 \
    --batch_size 2 \
    --multi_gpu

2026-01-17 11:57:26.430577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768651046.585011      68 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768651046.629051      68 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768651046.980763      68 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768651046.980828      68 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768651046.980835      68 computation_placer.cc:177] computation placer alr

In [ ]:
!cd /kaggle/working/SpineSegmentation_HealthiVert && python evaluate.py \
    --data_dir /kaggle/input/verse-19-3d-images \
    --output_dir /kaggle/working/output \
    --splits_dir . \
    --n_folds 5

Using device: cuda


In [10]:
# !cd /kaggle/working/SpineSegmentation_HealthiVert && python run_inference.py \
#   --input /kaggle/working/verse19/dataset-verse19test/rawdata/sub-verse012/sub-verse012_ct.nii.gz \
#   --output /kaggle/working/verse19/inference_output \
#   --stage all \
#   --stage1_ckpt /kaggle/working/verse19/output/stage1/fold_0/checkpoints/fold_0/final_model.pth \
#   --stage2_ckpt /kaggle/working/verse19/output/stage2/fold_0/checkpoints/fold_0/final_model.pth \
#   --stage3_ckpt /kaggle/working/verse19/output/stage3/fold_0/checkpoints/fold_0/final_model.pth